<a href="https://colab.research.google.com/github/rodrigobernall/ds4a_group30/blob/master/scripts/EDA/origin_destination_matrix_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando las librerías

In [0]:
!pip install geopandas

In [0]:
import pandas as pd
import geopandas as gpd
import folium

# Importando los datos

In [0]:
url = 'https://www.dropbox.com/s/7ixw7pe46q3theo/green_trips_nta.csv.gz?dl=1'
green_trips_nta = pd.read_csv(url, compression='gzip')
#green_trips_nta = green_trips_nta.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

In [0]:
url = 'https://www.dropbox.com/s/k4amamqkqf7ap92/yellow_trips_nta.csv.gz?dl=1'
yellow_trips_nta = pd.read_csv(url, compression='gzip')

In [0]:
url = 'https://www.dropbox.com/s/pjovxyr8w6x6y5x/zones.csv?dl=1'
zones = pd.read_csv(url)
#zones

In [0]:
url = 'https://www.dropbox.com/s/8gys15vzjpd1ibu/demographics.csv?dl=1'
demographics = pd.read_csv(url)
#demographics

## Computing NTA centroids

In [0]:
url = 'https://www.dropbox.com/s/pt2qx83gxzpsacp/nta.json?dl=1'
nta_polygons = gpd.read_file(url)

In [0]:
url = 'https://www.dropbox.com/s/3ts67qym904tess/nta_centroids.json?dl=1'
nta_centroids = gpd.read_file(url)

## Origin-destination matrices

In [0]:
most_common_destinations_green = green_trips_nta['dropoff_nta'].value_counts()
most_common_origins_green = green_trips_nta['pickup_nta'].value_counts()

In [0]:
ODM_green = green_trips_nta.groupby(['pickup_nta', 'dropoff_nta']).size()
ODM_green = pd.DataFrame(ODM_green)
ODM_green = ODM_green.sort_values(by=[0], ascending=False)
ODM_green.columns = ['trips']

Adding the centroids of the NTAs to the ODM (Origin-Destination Matrix)

In [12]:
a = ODM_green.reset_index()
a = a.merge(nta_centroids, left_on='pickup_nta', right_on='id')
a = a.merge(nta_centroids, left_on='dropoff_nta', right_on='id', suffixes=('_pickup', '_dropoff'))
a = a.drop(columns=['id_pickup', 'id_dropoff'])
a

,pickup_nta,dropoff_nta,trips,geometry_pickup,geometry_dropoff
0,QN70,QN70,41981,POINT (40.76150 -73.91970),POINT (40.76150 -73.91970)
1,BK73,QN70,2499,POINT (40.71492 -73.95863),POINT (40.76150 -73.91970)
2,QN28,QN70,5973,POINT (40.75732 -73.88532),POINT (40.76150 -73.91970)
3,MN11,QN70,279,POINT (40.80434 -73.95129),POINT (40.76150 -73.91970)
4,BK37,QN70,303,POINT (40.67124 -73.98390),POINT (40.76150 -73.91970)
5,MN09,QN70,221,POINT (40.80789 -73.96254),POINT (40.76150 -73.91970)
6,MN33,QN70,455,POINT (40.79002 -73.94575),POINT (40.76150 -73.91970)
7,MN03,QN70,206,POINT (40.81827 -73.94077),POINT (40.76150 -73.91970)
8,QN29,QN70,3519,POINT (40.73950 -73.87712),POINT (40.76150 -73.91970)
9,QN17,QN70,567,POINT (40.72144 -73.84767),POINT (40.76150 -73.91970)


In [0]:
def create_paths_point(point_pickup, point_dropoff):
    pickup = tuple([point_pickup.x, point_pickup.y])
    dropoff = tuple([point_dropoff.x, point_dropoff.y])
    points = [pickup, dropoff]
    return points

def create_paths_dataframe(dataframe):
    long_pickup = dataframe['geometry_pickup'].apply(lambda p: p.y)
    lat_pickup = dataframe['geometry_pickup'].apply(lambda p: p.x)
    
    long_dropoff = dataframe['geometry_dropoff'].apply(lambda p: p.y)
    lat_dropoff = dataframe['geometry_dropoff'].apply(lambda p: p.x)

    point_pickup = zip(lat_pickup, long_pickup)
    point_dropoff = zip(lat_dropoff, long_dropoff)

    path = zip(point_pickup, point_dropoff)
    
    paths = []
    for i in path:
        paths.append(list(i))

    return paths

In [0]:
paths_greens = create_paths_dataframe(a.sort_values(by='trips', ascending=False))

In [17]:
a.sort_values(by='trips', ascending=False)

,pickup_nta,dropoff_nta,trips,geometry_pickup,geometry_dropoff
0,QN70,QN70,41981,POINT (40.76150 -73.91970),POINT (40.76150 -73.91970)
1293,BK73,BK73,40643,POINT (40.71492 -73.95863),POINT (40.71492 -73.95863)
1991,BK73,BK76,31137,POINT (40.71492 -73.95863),POINT (40.72951 -73.94953)
582,QN28,QN28,27368,POINT (40.75732 -73.88532),POINT (40.75732 -73.88532)
4882,MN11,MN03,26780,POINT (40.80434 -73.95129),POINT (40.81827 -73.94077)
4556,BK37,BK37,26568,POINT (40.67124 -73.98390),POINT (40.67124 -73.98390)
3144,MN09,MN12,24967,POINT (40.80789 -73.96254),POINT (40.79039 -73.97391)
3941,MN33,MN34,24505,POINT (40.79002 -73.94575),POINT (40.80118 -73.93735)
4886,MN03,MN03,23506,POINT (40.81827 -73.94077),POINT (40.81827 -73.94077)
588,QN29,QN28,23384,POINT (40.73950 -73.87712),POINT (40.75732 -73.88532)


In [16]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="Stamen Toner")


for path in paths_greens[:500]:
    folium.PolyLine(path, color="green", weight=1.5).add_to(folium_map)

#Now we can have a scatter plot of the first 1000 data points on the above map
#for i in range(0,1000):
#    marker = folium.CircleMarker(location=[listings["latitude"][i],listings["longitude"][i]],radius=5,color="r",fill=True)
#    marker.add_to(folium_map)

folium_map